In [8]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import HuggingFaceHub
from langchain_google_genai import GoogleGenerativeAI

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
llm = GoogleGenerativeAI(model="models/text-bison-001")
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

## **Pros of Python**

* **Easy to learn:** Python is a relatively easy language to learn, especially for those with experience in other programming languages. The syntax is simple and straightforward, and there are a number of resources available to help you get started.
* **Versatile:** Python can be used for a wide variety of tasks, including web development, data science, and machine learning. This versatility makes it a good choice for developers who want to work in a variety of fields.
* **Open source:** Python is an open source language, which means that it is free to use and modify. This can be a major advantage for developers who want to be able to customize the language to their specific needs.
* **Large community:** Python has a large and active community of developers who are constantly creating new libraries and tools. This can be a valuable resource for developers who are looking for help or inspiration.

## **Cons of Python**

* **Performance:** Python is not as fast as 

In [11]:
# First we make our text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)

In [12]:
unit1 = "Data/UNIT-1.pdf"
unit2 = "Data/UNIT-II.pdf"
unit3 = "Data/UNIT III.pdf"
unit4 = "Data/UNIT IV.pdf"
unit5 = "Data/UNIT-V.pdf"

In [13]:
loader = PyPDFLoader(unit1)
pages = loader.load_and_split()
len(pages)

11

In [14]:
pages[0].page_content

'CS8791: Cloud Computing  Department of CSE   \nKCG College of Technology   1  \n CS8791  CLOUD  COMPUTING  L T  P C \n3 0 0  3 \nUNIT I INTRODUCTION (9)  \nIntroduction to Cloud Computing – Definition of Cloud – Evolution of Cloud Computing – \nUnderlying Principles of Parallel and Distributed Computing – Cloud Characteristics – Elasticity \nin Cloud – On- demand Provisioning.  \n \n \nCloud computing consists of three distinct types of computing services delivered remotely to \nclients via the internet. Clients typically pay a monthly or annual service fee to providers, to gain \naccess to systems that deliver software as a service, platforms as a service and infrastructure as a \nservice to subscribers . Cloud Computing is the delivery of computing services such as servers, \nstorage, databases, networking, software, analytics, intelligence, and more, over the Cloud.  \n \nA Brief History of Cloud Computing  \nWhen we think of the cloud, rarely do we cast our minds back to times bef

In [15]:
# Then we split our user information into different documents
docs = text_splitter.split_documents(pages)

In [16]:
# Let's see how many documents we created
len(docs)

17

Because we have a special requset for the LLM on our data, I want to make custom prompts. This will allow me to tinker with what data the LLM pulls out. I'll use Langchain's `load_summarize_chain` with custom prompts to do this. We aren't making a summary, but rather just using `load_summarize_chain` for its easy mapreduce functionality.

First let's make our custom map prompt. This is where we'll instruction the LLM that it will pull out interview questoins and what makes a good question.

In [17]:
map_prompt = """You are a helpful AI bot that aids a teacher in question selection for standardized tests.
Below is information about the chapter.
Information will include contents about the chapter that a student is taught by a teacher.
Your goal is to generate question which can be asked to a student learning this chapter in a test.

% START OF INFORMATION ABOUT chapter:
{text}
% END OF INFORMATION ABOUT chapter:

Please respond with list of a few questions based on the topics above

YOUR RESPONSE:"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

Then we'll make our custom combine promopt. This is the set of instructions that we'll LLM on how to handle the list of questions that is returned in the first step above.

In [18]:
combine_prompt = """
You are a helpful AI bot that aids a teacher in question selection for standardized tests.
You will be given a list of potential questions that we can ask students.

Please consolidate the questions and return a list of questions.

% QUESTIONS
{text}
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

Let's create our LLM and chain. I'm increasing the color a bit for more creative language. If you notice that your questions have hallucinations in them, turn temperature to 0

In [19]:
# llm = HuggingFaceHub(
#     repo_id="HuggingFaceH4/zephyr-7b-beta",
#     task="text-generation",
#     model_kwargs={
#         "temperature": 0.1,
#         "repetition_penalty": 1.03,
#     },
# )

#llm = ChatOpenAI(temperature=.25, openai_api_key = openai_api_key)

chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=map_prompt_template,
                             combine_prompt=combine_prompt_template,
#                              verbose=True
                            )

In [20]:
output = chain({"input_documents": docs})

c:\Users\Tushar\OneDrive\Desktop\langchain\langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [21]:
print (output['output_text'])


1. What is cloud computing?
2. What are the three main types of cloud computing?
3. What are the benefits of cloud computing?
4. What are the challenges of cloud computing?
5. What are the different cloud providers?
6. How do you choose a cloud provider?
7. How do you migrate to the cloud?
8. What are the future trends in cloud computing?
9. What is the difference between cloud computing and grid computing?
10. What is the difference between cloud computing and distributed computing?

1. What is cloud computing?
2. What are the benefits of cloud computing?
3. What are the challenges of cloud computing?
4. What are the different types of cloud computing?
5. What are the different cloud providers?
6. How do you choose a cloud provider?
7. How do you migrate to the cloud?
8. What are the future trends in cloud computing?
9. What are the different security challenges of cloud computing?
10. What are the different compliance challenges of cloud computing?
